when you clone these github repos, you can go to default_configs.py file to change training parameters. I used ILQL here but i also tried PPO. PPO is better 🥇.

**Clone the Repository**:

The command !git clone https://github.com/CarperAI/trlx.git clones the trlx repository from GitHub into the local directory. This repository contains the code for training the model.

In [1]:
!git clone https://github.com/CarperAI/trlx.git
!git config --global --add safe.directory /content/trlx && cd /content/trlx && pip install -e .

Cloning into 'trlx'...
remote: Enumerating objects: 7589, done.
remote: Counting objects: 100% (2874/2874), done.
remote: Compressing objects: 100% (599/599), done.
remote: Total 7589 (delta 2486), reused 2460 (delta 2275), pack-reused 4715
Receiving objects: 100% (7589/7589), 46.76 MiB | 19.23 MiB/s, done.
Resolving deltas: 100% (5184/5184), done.
Obtaining file:///content/trlx
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.9/809.9 kB 55.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

**Setup for the Repository**:
The command !git config --global --add safe.directory /content/trlx && cd /content/trlx && pip install -e . sets the working directory to /content/trlx and installs the package and all its dependencies in editable mode (pip install -e .). This means any changes made in the code will be immediately effective without the need to reinstall the package.

In [2]:
# uninstall scikit_learn + jax to avoid numpy issues
!pip uninstall -y scikit_learn jax

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: jax 0.4.13
Uninstalling jax-0.4.13:
  Successfully uninstalled jax-0.4.13


**Change Directory**:

The os.chdir('/content/trlx') changes the current working directory to the cloned trlx repository.

In [3]:
import os

# run within repo
os.chdir('/content/trlx')
print(os.getcwd())

/content/trlx


**Import Modules**:

Several Python modules and functions are imported, such as load_dataset from the datasets library, pipeline from the transformers library, and several from the trlx package that was just cloned.

In [4]:
import yaml
from datasets import load_dataset
from transformers import pipeline
import pathlib
from typing import Dict, List
import trlx
from trlx.data.default_configs import TRLConfig, default_ilql_config

[2023-07-15 16:51:35,235] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


If we have trained our reward model with our data we could have used it as well. Since not, we are only using the transfer learning model with a random reward model for sentiments.

**Configuration for Training**:

The default_ilql_config function is called to fetch the default configuration for training the model using the Incremental Lagrangian Q-Learning (ILQL) method. Several parameters are then modified in this configuration, including disabling the tracker, setting the batch size to 16, the number of epochs to 10, and changing the model path to use the AliChazz/GPT2_Fine_Tune_Requirement_Produce model.

In [5]:
default_config = default_ilql_config().to_dict()
default_config['train']['tracker'] = None
default_config['train']['batch_size'] = 16
default_config['train']['epochs'] = 10
default_config['model']['model_path']='AliChazz/GPT2_Fine_Tune_Requirement_Produce'
config = TRLConfig.update(default_config, {})
print(config)

{
    "method": {
        "name": "ilqlconfig",
        "tau": 0.7,
        "gamma": 0.99,
        "cql_scale": 0.1,
        "awac_scale": 1,
        "alpha": 0.001,
        "beta": 0,
        "steps_for_target_q_sync": 5,
        "two_qs": true,
        "gen_kwargs": {
            "max_new_tokens": 56,
            "top_k": 20,
            "beta": 1,
            "temperature": 1.0
        }
    },
    "model": {
        "model_path": "AliChazz/GPT2_Fine_Tune_Requirement_Produce",
        "model_arch_type": "causal",
        "num_layers_unfrozen": -1,
        "peft_config": null
    },
    "optimizer": {
        "name": "adamw",
        "kwargs": {
            "lr": 5e-05,
            "betas": [
                0.9,
                0.95
            ],
            "eps": 1e-08,
            "weight_decay": 1e-06
        }
    },
    "scheduler": {
        "name": "cosine_annealing",
        "kwargs": {
            "T_max": 1000000000000.0,
            "eta_min": 5e-05
        }
    },
   

Explainations for the code below :

This code is responsible for sentiment analysis using a pretrained DistilBERT model and defining the metric function to use during training.

Function - get_positive_score: The purpose of this function is to take the output of a sentiment analysis pipeline (which is a list of dictionaries) and return the score corresponding to the "POSITIVE" sentiment. Here is how it works:

It takes a list of dictionaries as input where each dictionary represents a sentiment (like "POSITIVE" or "NEGATIVE") and its corresponding score.
It applies a map function to each dictionary which transforms each dictionary into a tuple containing the sentiment and its score.
It then converts the resulting list of tuples back into a dictionary.
It then returns the score associated with the "POSITIVE" sentiment.
DistilBERT Sentiment Analysis Pipeline: This piece of code sets up a sentiment analysis pipeline using a pretrained DistilBERT model ("lvwerra/distilbert-imdb").

"sentiment-analysis" is the task type for the pipeline.
"lvwerra/distilbert-imdb" is the pretrained model which is fine-tuned on the IMDB dataset.
"top_k" indicates that the pipeline will return scores for the top 2 sentiments.
"truncation=True" ensures that inputs longer than the model's maximum input length are truncated.
"batch_size" sets the number of samples to process at a time.
"device=0" ensures that the pipeline uses the GPU if available.
Function - metric_fn: This function takes a list of text samples as input and returns a dictionary with the sentiment scores for those samples.

It applies the sentiment analysis pipeline to the samples to get the sentiment scores.
It then applies the get_positive_score function to extract the "POSITIVE" sentiment scores.
Finally, it returns a dictionary where the key is "sentiments" and the value is the list of "POSITIVE" sentiment scores.
This function will be used during training to calculate the reward for the RL algorithm: the higher the positive sentiment score for a generated text, the higher the reward.

In [6]:
def get_positive_score(scores):
    "Extract value associated with a positive sentiment from pipeline's output"
    return dict(map(lambda x: tuple(x.values()), scores))["POSITIVE"]

sentiment_fn = pipeline(
    "sentiment-analysis",
    "lvwerra/distilbert-imdb",
    top_k=2,
    truncation=True,
    batch_size=256,
    device=0,
)

def metric_fn(samples: List[str], **kwargs) -> Dict[str, List[float]]:
    sentiments = list(map(get_positive_score, sentiment_fn(samples)))
    return {"sentiments": sentiments}

imdb = load_dataset("imdb", split="train+test")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


In [7]:
trainer = trlx.train(
    samples=imdb["text"],
    rewards=imdb["label"],
    eval_prompts=[
        "I don't know much about Hungarian underground",
        "What made this movie so distinctly",
        "Like the sandwich I just bought at the grocery store,",
        "I cannot believe how much this movie made me want to"
    ] * 20,
    metric_fn=metric_fn,
    config=config,
)

[RANK 0] Initializing model: AliChazz/GPT2_Fine_Tune_Requirement_Produce


Using pad_token, but it is not set yet.
[RANK 0] Collecting rollouts
Token indices sequence length is longer than the specified maximum sequence length for this model (1169 > 1024). Running this sequence through the model will result in indexing errors
[RANK 0] Logging sample example


                                                  Sample Example                                                   
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Prompt        ┃ Response                                                                               ┃ Reward ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ <|endoftext|> │ I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that   │ 0      │
│               │ surrounded it when it was first released in 1967. I also heard that at first it was    │        │
│               │ seized by U.S. customs if it ever tried to enter this country, therefore being a fan   │        │
│               │ of films considered "                                                                  │        │
└───────────────┴────────────────────────────────────────────────────────────────────────────────────────┴────────┘

[RANK 0] Logging experience string statistics


     Experience String Stats (mean ∈ [min, max])      
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt Length ┃ Output Length   ┃ Sample Length    ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ 1.00 ∈ [1, 1] │ 62.76 ∈ [9, 63] │ 63.76 ∈ [10, 64] │
└───────────────┴─────────────────┴──────────────────┘

...

                                      Evaluation #9 metrics/sentiments: 0.442                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ prompt                                        ┃ output                                             ┃ sentiments ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ I don't know much about Hungarian underground │  and the underground is a great place to enjoy a   │ 0.989      │
│                                               │ good story.<br /><br />This movie starts like a    │            │
│                                               │ great story line and there is much that makes you  │            │
│                                               │ want to stay with it (which usually can cause you  │            │
│                                               │ to be in a bit of trouble as it seems there are    │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ What made this movie so distinctly            │  awful is that it had to be. I didn't care for the │ 0.00399    │
│                                               │ dialogue, it was awful. But the idea that the      │            │
│                                               │ character of the character is somehow a "big" fat  │            │
│                                               │ woman has to give me another reason to hate her.   │            │
│                                               │ All of the characters have an awful character,     │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ Like the sandwich I just bought at the        │  cinema was a very bad idea. The movie was just    │ 0.0127     │
│                                               │ too weak. In it, a bunch of guys with a dog walk   │            │
│                                               │ around talking like kids, making a bunch of        │            │
│                                               │ friends at that (which is why the kids don't show  │            │
│                                               │ up). I saw some of the movie and loved             │            │
└───────────────────────────────────────────────┴────────────────────────────────────────────────────┴────────────┘

[RANK 0] Saving intermediate checkpoint into ckpts/checkpoint_1000
[RANK 0] Evaluating model


[generation sweep 0/1 | eval batch 0/5]:   0%|          | 0/5 [00:00<?, ?it/s]

[RANK 0] Computing metrics
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
[RANK 0] Summarizing evaluation


                                     Evaluation #10 metrics/sentiments: 0.556                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ prompt                                        ┃ output                                             ┃ sentiments ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ I don't know much about Hungarian underground │  cinema but when i saw the film I thought it was   │ 0.995      │
│                                               │ really interesting and the director of it has to   │            │
│                                               │ do with the "The Man" and "The Man & The Machine". │            │
│                                               │ I'm quite shocked and am still amazed how it turns │            │
│                                               │ out. The story is well written and very original   │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ What made this movie so distinctly            │  different from the films that were made in the    │ 0.96       │
│                                               │ 70's, 80's & 90's is that when you are a child. It │            │
│                                               │ is the kind of thing that you can't do without,    │            │
│                                               │ and if you have to take someone into your family   │            │
│                                               │ at the young age, they will do                     │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ Like the sandwich I just bought at the        │  box office, is the most embarrassing piece of     │ 0.115      │
│                                               │ garbage I've ever seen. I just want my money back. │            │
│                                               │ It's so embarrassing. It's not funny. But it's     │            │
│                                               │ hilarious. I like the whole concept. It is so      │            │
│                                               │ embarrassing, so annoying... It's like having your │            │
└───────────────────────────────────────────────┴────────────────────────────────────────────────────┴────────────┘

You can see now our model is changed in a way to not output a requirement but a positive sentimental continuation.

**Generate Text**:

 Finally, the trained model is used to generate a text continuation for the prompt 'Aeroplanes shall work with'. The generated text is then printed to the console.

In [8]:
# output
input_str = 'Aeroplanes shall work with'
trainer_output = trainer.generate_eval(
    **trainer.tokenizer(input_str, return_tensors='pt'))[0]
print(trainer.tokenizer.decode(trainer_output))

Aeroplanes shall work with a group of people (at least in the U.S. in the late 1930's), and they are all like a group who are trying to get away from a town and to do so they are put into "bodies" and put into a car and driven to a
